In [78]:
sys.path.insert(0, "../../vca_perf/")

In [18]:
import sys
sys.path.insert(0, "../../vca_perf/qoe_estimation/")
import pandas as pd
import numpy as np
import json
import re
import ast
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os, glob
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from feature_e

SyntaxError: invalid syntax (1778528075.py, line 16)

In [84]:
%run "../scripts/helper_functions.py"
vca = "teams"
data_dir = "../../data/Data_11_9/*"
data = {}
ip = '192.168.1.187'
data[vca], stat = read_data(data_dir, vca, ip)

In [85]:
filelist = list(data[vca].keys())

In [97]:
feature_list = []
config = {"n_features_size": {"teams": 200, "meet": 300}, "n_features_iat": 200, "prediction_window": 1}
feature_extractor = FeatureExtractor(["LSTATS", "TSTATS"], config, vca)
webrtc_list = []

for fname in filelist:
    df_webrtc = WebRTCReader(fname).get_webrtc()
    df_net = data[vca][fname]['pcap']
    df_features = feature_extractor.extract_features(df_net)
    feature_list.append(df_features)
    df_webrtc["fname"] = fname
    webrtc_list.append(df_webrtc)

df_features = pd.concat(feature_list).reset_index()
df_webrtc = pd.concat(webrtc_list).reset_index()

In [98]:
df_merge = pd.merge(df_features, df_webrtc, left_on="et", right_on="ts")

In [96]:
df_merge

,index_x,et,l_max,l_mean,l_min,l_num_bytes,l_num_pkts,l_num_unique,l_q1,l_q2,...,framesReceived,[interFrameDelayStDev_in_ms],nackCount,packetsReceived,trackIdentifier,freezeCount*,totalFreezesDuration*,totalFramesDuration*,pauseCount*,totalPausesDuration*
0,0,1667883542,290,234.000000,203,2808,12,9,207.0,217.0,...,0.0,0.000000,0.0,0.0,3aa135e4-1a15-4995-8232-6d18aedc6825,0.0,0.0,0.000,0.0,0.0
1,1,1667883543,227,211.285714,199,1479,7,6,204.0,214.0,...,0.0,0.000000,0.0,0.0,3aa135e4-1a15-4995-8232-6d18aedc6825,0.0,0.0,0.000,0.0,0.0
2,2,1667883544,1181,534.000000,76,18690,35,16,304.0,304.0,...,6.0,57.868865,1.0,12.0,3aa135e4-1a15-4995-8232-6d18aedc6825,0.0,0.0,0.715,0.0,0.0
3,3,1667883545,1179,607.944444,76,32829,54,29,304.0,304.0,...,10.0,58.932249,2.0,35.0,3aa135e4-1a15-4995-8232-6d18aedc6825,0.0,0.0,1.013,0.0,0.0
4,4,1667883546,1209,637.106061,263,42049,66,37,304.0,836.0,...,15.0,16.646995,3.0,69.0,3aa135e4-1a15-4995-8232-6d18aedc6825,0.0,0.0,0.968,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6216,21,1667868838,1220,1173.473846,1096,381379,325,47,1145.0,1175.0,...,25.0,9.651701,222.0,4271.0,5da8accc-dbf1-4bc1-9207-4d155e48c9c5,0.0,0.0,1.045,0.0,0.0
6217,22,1667868839,1225,1171.338323,1135,391227,334,45,1148.0,1164.0,...,26.0,9.151408,236.0,4556.0,5da8accc-dbf1-4bc1-9207-4d155e48c9c5,0.0,0.0,0.955,0.0,0.0
6218,23,1667868840,1225,1179.517915,1103,362112,307,48,1152.0,1181.0,...,24.0,10.423358,248.0,4823.0,5da8accc-dbf1-4bc1-9207-4d155e48c9c5,0.0,0.0,1.014,0.0,0.0
6219,24,1667868841,1226,1184.690184,1109,386209,326,47,1168.0,1190.0,...,26.0,11.412274,261.0,5109.0,5da8accc-dbf1-4bc1-9207-4d155e48c9c5,0.0,0.0,0.995,0.0,0.0
